<a href="https://colab.research.google.com/github/afrahhassan/afrahhassan/blob/main/assignment2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
print (" hello")

 hello


In [ ]:
print ("hello from Chang")

hello from Chang


# **Reading Data**

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
!cp "/content/drive/MyDrive/ML/Assignment2" .

cp: -r not specified; omitting directory '/content/drive/MyDrive/ML/Assignment2'


In [10]:
!cp -r "/content/drive/MyDrive/ML/Assignment2" .


In [40]:
import zipfile


zip_file_path = '/content/drive/My Drive/ML/Assignment2/trafficsigns_dataset.zip'

extract_to_path = '/content/trafficsigns_dataset'


with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to_path)

print("Extraction completed!")


Extraction completed!


In [51]:
import os
import csv

root_dir = '/content/trafficsigns_dataset'

output_csv = '/content/drive/My Drive/ML/Assignment2/images.csv'

with open(output_csv, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['image_path', 'class'])

    for root, dirs, files in os.walk(root_dir):
        for file in files:
            if file.endswith(('.png', '.jpg', '.jpeg')):
                full_path = os.path.join(root, file)
                class_name = os.path.basename(root)
                writer.writerow([full_path, class_name])
                print(f"Added: {file} - Class: {class_name}")
            else:
                print(f"Skipped: {file}")


print("CSV file has been created successfully.")


Streaming output truncated to the last 5000 lines.
Added: 01332_00000.png - Class: parking
Added: 00175_00001.png - Class: parking
Added: 00177_00001.png - Class: parking
Added: 01637_00002.png - Class: parking
Added: 00851_00000.png - Class: parking
Added: 01192_00000.png - Class: parking
Added: 00106_00000.png - Class: parking
Added: 00477_00002.png - Class: parking
Added: 00113_00001.png - Class: parking
Added: 00054_00002.png - Class: parking
Added: 01694_00002.png - Class: parking
Added: 00035_00001.png - Class: parking
Added: 00321_00001.png - Class: parking
Added: 00495_00002.png - Class: parking
Added: 00036_00001.png - Class: parking
Added: 00062_00002.png - Class: parking
Added: 00615_00001.png - Class: parking
Added: 00827_00000.png - Class: parking
Added: 00486_00001.png - Class: parking
Added: 00458_00002.png - Class: parking
Added: 00804_00000.png - Class: parking
Added: 01039_00001.png - Class: giveway
Added: 01038_00000.png - Class: giveway
Added: 01891_00001.png - Clas